In [9]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

KERALA STATE BUS_DETAILS

In [6]:

df1 = pd.read_csv(r'C:\Users\praba\Desktop\red_bus_project\webscrap_busroute_names_links\kerala_routes.csv')
df1

,Route Name,Route Link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
5,Bangalore to Kannur (Kerala),https://www.redbus.in/bus-tickets/bangalore-to...
6,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
7,Bangalore to Kalpetta (kerala),https://www.redbus.in/bus-tickets/bangalore-to...
8,Kalpetta (kerala) to Bangalore,https://www.redbus.in/bus-tickets/kalpetta-to-...
9,Kannur (Kerala) to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...


In [9]:
print(df1.columns)

Index(['Route Name', 'Route Link'], dtype='object')


In [13]:
# Initialize the WebDriver
driver_k = webdriver.Chrome()

# Lists to store data
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

# Assuming 'df1' contains the route details
for i, row in df1.iterrows():
    route_link = row["Route Link"]
    route_name = row["Route Name"]
    
    # Open the route page
    driver_k.get(route_link)
    time.sleep(3)  # wait for the page to load
    
    # Try to click the "View Buses" button if present
    try:
        WebDriverWait(driver_k, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        ).click()
        time.sleep(3)
    except Exception as e:
        print(f"Error clicking 'View Buses' button for {route_name}: {e}")
    
    # Scroll to the bottom of the page and keep checking for new content
    old_page_source = driver_k.page_source
    scroll_attempts = 0
    while True:
        driver_k.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)  # Adjust wait time to ensure the page has time to load new elements
        
        new_page_source = driver_k.page_source
        
        # Check if new buses have been loaded, based on changes in the page source
        if new_page_source == old_page_source:
            scroll_attempts += 1
            if scroll_attempts > 5:  # Stop after a few unsuccessful attempts
                break
        else:
            old_page_source = new_page_source
            scroll_attempts = 0  # Reset attempts if new content was loaded
    
    # Extract bus details
    bus_elements = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type_elements = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    departure_elements = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    arrival_elements = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration_elements = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price_elements = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seat_elements = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    rating_elements = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    
    # Log how many buses were found
    print(f"Found {len(bus_elements)} buses on {route_name} page")
    
    # Append data for each bus
    for bus, bus_type, dep, arr, dur, price, seat, rating in zip(bus_elements, bus_type_elements, departure_elements, arrival_elements, duration_elements, price_elements, seat_elements, rating_elements):
        Bus_names_k.append(bus.text)
        Bus_types_k.append(bus_type.text)
        Departure_k.append(dep.text)
        Arrival_k.append(arr.text)
        Total_Duration_k.append(dur.text)
        Prices_k.append(price.text)
        Seats_Available_k.append(seat.text)
        Ratings_k.append(rating.text)
        Route_names.append(route_name)
        Route_links.append(route_link)

# Create a DataFrame from the collected data
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses = pd.DataFrame(data)

# Save the data to a CSV file
df_buses.to_csv("kerala_bus_details.csv", index=False)

# Close the WebDriver
driver_k.quit()

print("Data scraping completed and saved to kerala_bus_details.csv")



Found 52 buses on Bangalore to Kozhikode page
Found 40 buses on Kozhikode to Bangalore page
Found 83 buses on Kozhikode to Ernakulam page
Found 85 buses on Ernakulam to Kozhikode page
Found 53 buses on Mysore to Kozhikode page
Found 33 buses on Bangalore to Kannur (Kerala) page
Found 50 buses on Kozhikode to Mysore page
Found 49 buses on Bangalore to Kalpetta (kerala) page
Found 39 buses on Kalpetta (kerala) to Bangalore page
Found 29 buses on Kannur (Kerala) to Bangalore page
Found 13 buses on Kozhikode to Thiruvananthapuram page
Found 28 buses on Kozhikode to Thrissur page
Found 13 buses on Thiruvananthapuram to Kozhikode page
Found 46 buses on Kottayam to Kozhikode page
Found 43 buses on Kozhikode to Kottayam page
Data scraping completed and saved to bus_details.csv


ANDHRAPRADESH STATE BUS DETAILS

In [2]:
df2 = pd.read_csv(r'C:\Users\praba\Desktop\red_bus_project\webscrap_busroute_names_links\andhrapradesh_routes.csv')
df2

,Route Name,Route Link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...
4,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
5,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
6,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...
7,Hyderabad to Narasaraopet,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
9,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...


In [3]:
# Initialize the WebDriver
driver_k = webdriver.Chrome()

# Lists to store data
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []


for i, row in df2.iterrows():
    route_link = row["Route Link"]
    route_name = row["Route Name"]
    
    # Open the route page
    driver_k.get(route_link)
    time.sleep(3)  # wait for the page to load
    
    # Try to click the "View Buses" button if present
    try:
        WebDriverWait(driver_k, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        ).click()
        time.sleep(3)
    except Exception as e:
        print(f"Error clicking 'View Buses' button for {route_name}: {e}")
    
    # Scroll to the bottom of the page and keep checking for new content
    old_page_source = driver_k.page_source
    scroll_attempts = 0
    while True:
        driver_k.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)  # Adjust wait time to ensure the page has time to load new elements
        
        new_page_source = driver_k.page_source
        
        # Check if new buses have been loaded, based on changes in the page source
        if new_page_source == old_page_source:
            scroll_attempts += 1
            if scroll_attempts > 5:  # Stop after a few unsuccessful attempts
                break
        else:
            old_page_source = new_page_source
            scroll_attempts = 0  # Reset attempts if new content was loaded
    
    # Extract bus details
    bus_elements = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type_elements = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    departure_elements = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    arrival_elements = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    duration_elements = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price_elements = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seat_elements = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    rating_elements = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    
    # Log how many buses were found
    print(f"Found {len(bus_elements)} buses on {route_name} page")
    
    # Append data for each bus
    for bus, bus_type, dep, arr, dur, price, seat, rating in zip(bus_elements, bus_type_elements, departure_elements, arrival_elements, duration_elements, price_elements, seat_elements, rating_elements):
        Bus_names_k.append(bus.text)
        Bus_types_k.append(bus_type.text)
        Departure_k.append(dep.text)
        Arrival_k.append(arr.text)
        Total_Duration_k.append(dur.text)
        Prices_k.append(price.text)
        Seats_Available_k.append(seat.text)
        Ratings_k.append(rating.text)
        Route_names.append(route_name)
        Route_links.append(route_link)

# Create a DataFrame from the collected data
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Ratings": Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses = pd.DataFrame(data)

# Save the data to a CSV file
df_buses.to_csv("Andhrapradesh_bus_details.csv", index=False)

# Close the WebDriver
driver_k.quit()

print("Data scraping completed and saved to Andhrapradesh_bus_details.csv")

Found 528 buses on Hyderabad to Vijayawada page
Found 309 buses on Vijayawada to Hyderabad page
Found 126 buses on Hyderabad to Ongole page
Found 136 buses on Ongole to Hyderabad page
Found 61 buses on Kakinada to Visakhapatnam page
Found 277 buses on Bangalore to Tirupati page
Found 52 buses on Bangalore to Kadapa page
Found 50 buses on Hyderabad to Narasaraopet page
Found 42 buses on Chittoor (Andhra Pradesh) to Bangalore page
Found 74 buses on Visakhapatnam to Kakinada page
Found 54 buses on Narasaraopet to Hyderabad page
Found 122 buses on Bangalore to Chittoor (Andhra Pradesh) page
Found 51 buses on Kadapa to Bangalore page
Found 194 buses on Bangalore to Anantapur (andhra pradesh) page
Found 27 buses on Bangalore to Madanapalli page
Found 40 buses on Hyderabad to Vinukonda page
Found 9 buses on Madanapalli to Bangalore page
Found 262 buses on Hyderabad to Eluru page
Found 37 buses on Vinukonda to Hyderabad page
Found 34 buses on Anantapur (andhra pradesh) to Bangalore page
Found 

GUJARAT STATE BUS DETAILS

In [41]:
df3 = pd.read_csv(r"C:\Users\praba\Desktop\red_bus_project\webscrap_busroute_names_links\gujarat.csv")
df3

,Route Name,Route Link
0,Ahmedabad to Ganganagar (Sri Ganganagar),https://www.redbus.in/bus-tickets/ahmedabad-to...
1,Sirohi (Rajasthan) to Bikaner,https://www.redbus.in/bus-tickets/sirohi-to-bi...
2,Ahmedabad to Bikaner,https://www.redbus.in/bus-tickets/ahmedabad-to...
3,Jodhpur to Bikaner,https://www.redbus.in/bus-tickets/jodhpur-to-b...
4,Jodhpur to Hanumangarh,https://www.redbus.in/bus-tickets/jodhpur-to-h...
5,Sumerpur (Rajasthan) to Bikaner,https://www.redbus.in/bus-tickets/sumerpur-to-...
6,Jodhpur to Suratgarh,https://www.redbus.in/bus-tickets/jodhpur-to-s...
7,Abu Road to Bikaner,https://www.redbus.in/bus-tickets/abu-road-to-...
8,Ahmedabad to Suratgarh,https://www.redbus.in/bus-tickets/ahmedabad-to...
9,Sirohi (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/sirohi-to-jo...


In [42]:
# Initialize WebDriver
driver_k = webdriver.Chrome()

# Initialize lists for data storage
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

# Iterate through each row in DataFrame containing route links and names
for i, r in df3.iterrows():
    link = r["Route Link"]
    routes = r["Route Name"]

    # Navigate to the route link
    driver_k.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    try:
        elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)
    except Exception as e:
        print(f"Error clicking elements for {routes}: {e}")
        continue

    # Click "View Buses" button
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except Exception as e:
        print(f"Error clicking 'View Buses' button for {routes}: {e}")
        continue
    time.sleep(2)

    # Scroll to load all buses
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver_k.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except Exception as e:
        print(f"Error extracting ratings for {routes}: {e}")
        rating = []

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

print("Data extraction completed.")

Error clicking 'View Buses' button for Ahmedabad to Ganganagar (Sri Ganganagar): Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='button']"}
  (Session info: chrome=131.0.6778.206); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF687EF80D5+2992373]
	(No symbol) [0x00007FF687B8BFD0]
	(No symbol) [0x00007FF687A2590A]
	(No symbol) [0x00007FF687A7926E]
	(No symbol) [0x00007FF687A7955C]
	(No symbol) [0x00007FF687AC27D7]
	(No symbol) [0x00007FF687A9F3AF]
	(No symbol) [0x00007FF687ABF584]
	(No symbol) [0x00007FF687A9F113]
	(No symbol) [0x00007FF687A6A918]
	(No symbol) [0x00007FF687A6BA81]
	GetHandleVerifier [0x00007FF687F56A2D+3379789]
	GetHandleVerifier [0x00007FF687F6C32D+3468109]
	GetHandleVerifier [0x00007FF687F60043+3418211]
	GetHandleVerifier [0x00007FF687CEC78B+847787]
	(No symbol) [0x00007FF687B9757F]
	

In [48]:
# Debug: Print the lengths of all lists
print("Lengths of all lists:")
print(f"Bus_names_k: {len(Bus_names_k)}")
print(f"Bus_types_k: {len(Bus_types_k)}")
print(f"Departure_k: {len(Departure_k)}")
print(f"Arrival_k: {len(Arrival_k)}")
print(f"Total_Duration_k: {len(Total_Duration_k)}")
print(f"Prices_k: {len(Prices_k)}")
print(f"Seats_Available_k: {len(Seats_Available_k)}")
print(f"Ratings_k: {len(Ratings_k)}")
print(f"Route_names: {len(Route_names)}")
print(f"Route_links: {len(Route_links)}")

# Ensure all lists have the same length
min_length = min(
    len(Bus_names_k), len(Bus_types_k), len(Departure_k), len(Arrival_k),
    len(Total_Duration_k), len(Prices_k), len(Seats_Available_k),
    len(Ratings_k), len(Route_names), len(Route_links)
)

# Truncate lists to the minimum length
Bus_names_k = Bus_names_k[:min_length]
Bus_types_k = Bus_types_k[:min_length]
Departure_k = Departure_k[:min_length]
Arrival_k = Arrival_k[:min_length]
Total_Duration_k = Total_Duration_k[:min_length]
Prices_k = Prices_k[:min_length]
Seats_Available_k = Seats_Available_k[:min_length]
Ratings_k = Ratings_k[:min_length]
Route_names = Route_names[:min_length]
Route_links = Route_links[:min_length]

# Create a DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    'Seats_Available': Seats_Available_k,
    'Ratings': Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

try:
    df_gujarat_3 = pd.DataFrame(data)
    print(f"DataFrame created successfully with {df_gujarat_3.shape[0]} rows.")
except ValueError as e:
    print(f"Error creating DataFrame: {e}")

# Save to CSV
df_gujarat_3.to_csv("gujarat_bus_details.csv", index=False)
print("DataFrame saved to 'gujarat_bus_details.csv'")

Lengths of all lists:
Bus_names_k: 9
Bus_types_k: 9
Departure_k: 9
Arrival_k: 9
Total_Duration_k: 9
Prices_k: 9
Seats_Available_k: 9
Ratings_k: 9
Route_names: 9
Route_links: 9
DataFrame created successfully with 9 rows.
DataFrame saved to 'gujarat_bus_details.csv'


BIHAR STATE BUS DETAILS

In [24]:
df4 = pd.read_csv(r"C:\Users\praba\Desktop\red_bus_project\webscrap_busroute_names_links\bihar_routes.csv")
df4

,Route Name,Route Link
0,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
1,Patna (Bihar) to Motihari,https://www.redbus.in/bus-tickets/patna-to-mot...
2,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
3,Bettiah to Patna (Bihar),https://www.redbus.in/bus-tickets/bettiah-to-p...
4,Motihari to Delhi,https://www.redbus.in/bus-tickets/motihari-to-...
5,Delhi to Motihari,https://www.redbus.in/bus-tickets/delhi-to-mot...
6,Patna (Bihar) to Balmiki Nagar (bihar),https://www.redbus.in/bus-tickets/patna-to-bal...
7,Patna (Bihar) to Katihar,https://www.redbus.in/bus-tickets/patna-to-kat...
8,Balmiki Nagar (bihar) to Patna (Bihar),https://www.redbus.in/bus-tickets/balmiki-naga...
9,Patna (Bihar) to Ranchi,https://www.redbus.in/bus-tickets/patna-to-ranchi


In [ ]:
# Initialize WebDriver
driver_k = webdriver.Chrome()

# Initialize lists for data storage
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

# Iterate through each row in DataFrame containing route links and names
for i, r in df4.iterrows():
    link = r["Route Link"]
    routes = r["Route Name"]

    # Navigate to the route link
    driver_k.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    try:
        elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)
    except Exception as e:
        print(f"Error clicking elements for {routes}: {e}")
        continue

    # Click "View Buses" button
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except Exception as e:
        print(f"Error clicking 'View Buses' button for {routes}: {e}")
        continue
    time.sleep(2)

    # Scroll to load all buses
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver_k.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except Exception as e:
        print(f"Error extracting ratings for {routes}: {e}")
        rating = []

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

print("Data extraction completed.")
driver_k.quit()




Successfully Completed


In [37]:
# Debug: Print the lengths of all lists
print("Lengths of all lists:")
print(f"Bus_names_k: {len(Bus_names_k)}")
print(f"Bus_types_k: {len(Bus_types_k)}")
print(f"Departure_k: {len(Departure_k)}")
print(f"Arrival_k: {len(Arrival_k)}")
print(f"Total_Duration_k: {len(Total_Duration_k)}")
print(f"Prices_k: {len(Prices_k)}")
print(f"Seats_Available_k: {len(Seats_Available_k)}")
print(f"Ratings_k: {len(Ratings_k)}")
print(f"Route_names: {len(Route_names)}")
print(f"Route_links: {len(Route_links)}")

# Ensure all lists have the same length
min_length = min(
    len(Bus_names_k), len(Bus_types_k), len(Departure_k), len(Arrival_k),
    len(Total_Duration_k), len(Prices_k), len(Seats_Available_k),
    len(Ratings_k), len(Route_names), len(Route_links)
)

# Truncate lists to the minimum length
Bus_names_k = Bus_names_k[:min_length]
Bus_types_k = Bus_types_k[:min_length]
Departure_k = Departure_k[:min_length]
Arrival_k = Arrival_k[:min_length]
Total_Duration_k = Total_Duration_k[:min_length]
Prices_k = Prices_k[:min_length]
Seats_Available_k = Seats_Available_k[:min_length]
Ratings_k = Ratings_k[:min_length]
Route_names = Route_names[:min_length]
Route_links = Route_links[:min_length]

# Create a DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    'Seats_Available': Seats_Available_k,
    'Ratings': Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

try:
    df_bihar_4 = pd.DataFrame(data)
    print(f"DataFrame created successfully with {df_bihar_4.shape[0]} rows.")
except ValueError as e:
    print(f"Error creating DataFrame: {e}")

# Save to CSV
df_bihar_4.to_csv("bihar_bus_details.csv", index=False)
print("DataFrame saved to 'bihar_bus_details.csv'")

Lengths of all lists:
Bus_names_k: 4
Bus_types_k: 4
Departure_k: 4
Arrival_k: 4
Total_Duration_k: 4
Prices_k: 4
Seats_Available_k: 4
Ratings_k: 4
Route_names: 4
Route_links: 4
DataFrame created successfully with 4 rows.
DataFrame saved to 'bihar_bus_details.csv'


PUNJAB STATE BUS DETAILS

In [50]:
df5 = pd.read_csv(r"C:\Users\praba\Desktop\red_bus_project\webscrap_busroute_names_links\punjab.csv")
df5

,Route Name,Route Link
0,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
1,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
2,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
3,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...
4,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...
5,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...
6,Delhi Airport to Patiala,https://www.redbus.in/bus-tickets/delhi-airpor...
7,Jalandhar to Delhi,https://www.redbus.in/bus-tickets/jalandhar-to...
8,Delhi Airport to Ludhiana,https://www.redbus.in/bus-tickets/delhi-airpor...
9,Delhi to Jalandhar,https://www.redbus.in/bus-tickets/delhi-to-jal...


In [54]:
# Initialize WebDriver
driver_k = webdriver.Chrome()

# Initialize lists for data storage
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

# Iterate through each row in DataFrame containing route links and names
for i, r in df5.iterrows():
    link = r["Route Link"]
    routes = r["Route Name"]

    # Navigate to the route link
    driver_k.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    try:
        elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)
    except Exception as e:
        print(f"Error clicking elements for {routes}: {e}")
        continue

    # Click "View Buses" button
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except Exception as e:
        print(f"Error clicking 'View Buses' button for {routes}: {e}")
        continue
    time.sleep(2)

    # Scroll to load all buses
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver_k.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except Exception as e:
        print(f"Error extracting ratings for {routes}: {e}")
        rating = []

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

print("Data extraction completed.")
driver_k.quit()

Error clicking 'View Buses' button for Patiala to Delhi: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='button']"}
  (Session info: chrome=131.0.6778.206); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF687EF80D5+2992373]
	(No symbol) [0x00007FF687B8BFD0]
	(No symbol) [0x00007FF687A2590A]
	(No symbol) [0x00007FF687A7926E]
	(No symbol) [0x00007FF687A7955C]
	(No symbol) [0x00007FF687AC27D7]
	(No symbol) [0x00007FF687A9F3AF]
	(No symbol) [0x00007FF687ABF584]
	(No symbol) [0x00007FF687A9F113]
	(No symbol) [0x00007FF687A6A918]
	(No symbol) [0x00007FF687A6BA81]
	GetHandleVerifier [0x00007FF687F56A2D+3379789]
	GetHandleVerifier [0x00007FF687F6C32D+3468109]
	GetHandleVerifier [0x00007FF687F60043+3418211]
	GetHandleVerifier [0x00007FF687CEC78B+847787]
	(No symbol) [0x00007FF687B9757F]
	(No symbol) [0x00007FF68

In [55]:
# Debug: Print the lengths of all lists
print("Lengths of all lists:")
print(f"Bus_names_k: {len(Bus_names_k)}")
print(f"Bus_types_k: {len(Bus_types_k)}")
print(f"Departure_k: {len(Departure_k)}")
print(f"Arrival_k: {len(Arrival_k)}")
print(f"Total_Duration_k: {len(Total_Duration_k)}")
print(f"Prices_k: {len(Prices_k)}")
print(f"Seats_Available_k: {len(Seats_Available_k)}")
print(f"Ratings_k: {len(Ratings_k)}")
print(f"Route_names: {len(Route_names)}")
print(f"Route_links: {len(Route_links)}")

# Ensure all lists have the same length
min_length = min(
    len(Bus_names_k), len(Bus_types_k), len(Departure_k), len(Arrival_k),
    len(Total_Duration_k), len(Prices_k), len(Seats_Available_k),
    len(Route_names), len(Route_links)
)

# Handle missing Ratings data
if len(Ratings_k) < min_length:
    print("Filling missing Ratings data with 'NaN'.")
    Ratings_k.extend([None] * (min_length - len(Ratings_k)))

# Truncate all lists to the same length
Bus_names_k = Bus_names_k[:min_length]
Bus_types_k = Bus_types_k[:min_length]
Departure_k = Departure_k[:min_length]
Arrival_k = Arrival_k[:min_length]
Total_Duration_k = Total_Duration_k[:min_length]
Prices_k = Prices_k[:min_length]
Seats_Available_k = Seats_Available_k[:min_length]
Ratings_k = Ratings_k[:min_length]
Route_names = Route_names[:min_length]
Route_links = Route_links[:min_length]

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    'Seats_Available': Seats_Available_k,
    'Ratings': Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

# Create DataFrame and save to CSV
try:
    df_punjab = pd.DataFrame(data)
    print(f"DataFrame created successfully with {df_punjab.shape[0]} rows.")
    df_punjab.to_csv("punjab_bus_details.csv", index=False)
    print("DataFrame saved to 'punjab_bus_details.csv'")
except ValueError as e:
    print(f"Error creating DataFrame: {e}")


Lengths of all lists:
Bus_names_k: 52
Bus_types_k: 52
Departure_k: 52
Arrival_k: 52
Total_Duration_k: 52
Prices_k: 52
Seats_Available_k: 52
Ratings_k: 0
Route_names: 52
Route_links: 52
Filling missing Ratings data with 'NaN'.
DataFrame created successfully with 52 rows.
DataFrame saved to 'punjab_bus_details.csv'


RAJASTHAN STATE BUS DETAILS

In [56]:
df6 = pd.read_csv(r"C:\Users\praba\Desktop\red_bus_project\webscrap_busroute_names_links\rajasthan.csv")
df6

,Route Name,Route Link
0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
1,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
4,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...
7,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
8,Kota(Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kota-rajasth...
9,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...


In [57]:
# Initialize WebDriver
driver_k = webdriver.Chrome()

# Initialize lists for data storage
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

# Iterate through each row in DataFrame containing route links and names
for i, r in df6.iterrows():
    link = r["Route Link"]
    routes = r["Route Name"]

    # Navigate to the route link
    driver_k.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    try:
        elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)
    except Exception as e:
        print(f"Error clicking elements for {routes}: {e}")
        continue

    # Click "View Buses" button
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except Exception as e:
        print(f"Error clicking 'View Buses' button for {routes}: {e}")
        continue
    time.sleep(2)

    # Scroll to load all buses
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver_k.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except Exception as e:
        print(f"Error extracting ratings for {routes}: {e}")
        rating = []

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

print("Data extraction completed.")
driver_k.quit()

Error clicking 'View Buses' button for Udaipur to Jodhpur: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='button']"}
  (Session info: chrome=131.0.6778.206); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF687EF80D5+2992373]
	(No symbol) [0x00007FF687B8BFD0]
	(No symbol) [0x00007FF687A2590A]
	(No symbol) [0x00007FF687A7926E]
	(No symbol) [0x00007FF687A7955C]
	(No symbol) [0x00007FF687AC27D7]
	(No symbol) [0x00007FF687A9F3AF]
	(No symbol) [0x00007FF687ABF584]
	(No symbol) [0x00007FF687A9F113]
	(No symbol) [0x00007FF687A6A918]
	(No symbol) [0x00007FF687A6BA81]
	GetHandleVerifier [0x00007FF687F56A2D+3379789]
	GetHandleVerifier [0x00007FF687F6C32D+3468109]
	GetHandleVerifier [0x00007FF687F60043+3418211]
	GetHandleVerifier [0x00007FF687CEC78B+847787]
	(No symbol) [0x00007FF687B9757F]
	(No symbol) [0x00007FF

In [58]:
# Debug: Print the lengths of all lists
print("Lengths of all lists:")
print(f"Bus_names_k: {len(Bus_names_k)}")
print(f"Bus_types_k: {len(Bus_types_k)}")
print(f"Departure_k: {len(Departure_k)}")
print(f"Arrival_k: {len(Arrival_k)}")
print(f"Total_Duration_k: {len(Total_Duration_k)}")
print(f"Prices_k: {len(Prices_k)}")
print(f"Seats_Available_k: {len(Seats_Available_k)}")
print(f"Ratings_k: {len(Ratings_k)}")
print(f"Route_names: {len(Route_names)}")
print(f"Route_links: {len(Route_links)}")

# Ensure all lists have the same length
min_length = min(
    len(Bus_names_k), len(Bus_types_k), len(Departure_k), len(Arrival_k),
    len(Total_Duration_k), len(Prices_k), len(Seats_Available_k),
    len(Route_names), len(Route_links)
)

# Handle missing Ratings data
if len(Ratings_k) < min_length:
    print("Filling missing Ratings data with 'NaN'.")
    Ratings_k.extend([None] * (min_length - len(Ratings_k)))

# Truncate all lists to the same length
Bus_names_k = Bus_names_k[:min_length]
Bus_types_k = Bus_types_k[:min_length]
Departure_k = Departure_k[:min_length]
Arrival_k = Arrival_k[:min_length]
Total_Duration_k = Total_Duration_k[:min_length]
Prices_k = Prices_k[:min_length]
Seats_Available_k = Seats_Available_k[:min_length]
Ratings_k = Ratings_k[:min_length]
Route_names = Route_names[:min_length]
Route_links = Route_links[:min_length]

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    'Seats_Available': Seats_Available_k,
    'Ratings': Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

# Create DataFrame and save to CSV
try:
    df_rajasthan = pd.DataFrame(data)
    print(f"DataFrame created successfully with {df_rajasthan.shape[0]} rows.")
    df_rajasthan.to_csv("rajasthan_bus_details.csv", index=False)
    print("DataFrame saved to 'rajasthan_bus_details.csv'")
except ValueError as e:
    print(f"Error creating DataFrame: {e}")

Lengths of all lists:
Bus_names_k: 152
Bus_types_k: 152
Departure_k: 152
Arrival_k: 152
Total_Duration_k: 152
Prices_k: 152
Seats_Available_k: 152
Ratings_k: 35
Route_names: 152
Route_links: 152
Filling missing Ratings data with 'NaN'.
DataFrame created successfully with 152 rows.
DataFrame saved to 'rajasthan_bus_details.csv'


SIKKIM STATE BUS DETAILS

In [62]:
df7 = pd.read_csv(r"C:\Users\praba\Desktop\red_bus_project\webscrap_busroute_names_links\telangana_routes.csv")
df7

,Route Name,Route Link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
3,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
5,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
6,Hyderabad to Guntur (Andhra Pradesh),https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Sathupally,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Guntur (Andhra Pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/guntur-to-hy...
9,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...


In [63]:
# Initialize WebDriver
driver_k = webdriver.Chrome()

# Initialize lists for data storage
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

# Iterate through each row in DataFrame containing route links and names
for i, r in df7.iterrows():
    link = r["Route Link"]
    routes = r["Route Name"]

    # Navigate to the route link
    driver_k.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    try:
        elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)
    except Exception as e:
        print(f"Error clicking elements for {routes}: {e}")
        continue

    # Click "View Buses" button
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except Exception as e:
        print(f"Error clicking 'View Buses' button for {routes}: {e}")
        continue
    time.sleep(2)

    # Scroll to load all buses
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver_k.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except Exception as e:
        print(f"Error extracting ratings for {routes}: {e}")
        rating = []

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

print("Data extraction completed.")
driver_k.quit()

Error clicking 'View Buses' button for Hyderabad to Srisailam: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='button']"}
  (Session info: chrome=131.0.6778.206); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF687EF80D5+2992373]
	(No symbol) [0x00007FF687B8BFD0]
	(No symbol) [0x00007FF687A2590A]
	(No symbol) [0x00007FF687A7926E]
	(No symbol) [0x00007FF687A7955C]
	(No symbol) [0x00007FF687AC27D7]
	(No symbol) [0x00007FF687A9F3AF]
	(No symbol) [0x00007FF687ABF584]
	(No symbol) [0x00007FF687A9F113]
	(No symbol) [0x00007FF687A6A918]
	(No symbol) [0x00007FF687A6BA81]
	GetHandleVerifier [0x00007FF687F56A2D+3379789]
	GetHandleVerifier [0x00007FF687F6C32D+3468109]
	GetHandleVerifier [0x00007FF687F60043+3418211]
	GetHandleVerifier [0x00007FF687CEC78B+847787]
	(No symbol) [0x00007FF687B9757F]
	(No symbol) [0x000

In [64]:
# Debug: Print the lengths of all lists
print("Lengths of all lists:")
print(f"Bus_names_k: {len(Bus_names_k)}")
print(f"Bus_types_k: {len(Bus_types_k)}")
print(f"Departure_k: {len(Departure_k)}")
print(f"Arrival_k: {len(Arrival_k)}")
print(f"Total_Duration_k: {len(Total_Duration_k)}")
print(f"Prices_k: {len(Prices_k)}")
print(f"Seats_Available_k: {len(Seats_Available_k)}")
print(f"Ratings_k: {len(Ratings_k)}")
print(f"Route_names: {len(Route_names)}")
print(f"Route_links: {len(Route_links)}")

# Ensure all lists have the same length
min_length = min(
    len(Bus_names_k), len(Bus_types_k), len(Departure_k), len(Arrival_k),
    len(Total_Duration_k), len(Prices_k), len(Seats_Available_k),
    len(Route_names), len(Route_links)
)

# Handle missing Ratings data
if len(Ratings_k) < min_length:
    print("Filling missing Ratings data with 'NaN'.")
    Ratings_k.extend([None] * (min_length - len(Ratings_k)))

# Truncate all lists to the same length
Bus_names_k = Bus_names_k[:min_length]
Bus_types_k = Bus_types_k[:min_length]
Departure_k = Departure_k[:min_length]
Arrival_k = Arrival_k[:min_length]
Total_Duration_k = Total_Duration_k[:min_length]
Prices_k = Prices_k[:min_length]
Seats_Available_k = Seats_Available_k[:min_length]
Ratings_k = Ratings_k[:min_length]
Route_names = Route_names[:min_length]
Route_links = Route_links[:min_length]

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    'Seats_Available': Seats_Available_k,
    'Ratings': Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

# Create DataFrame and save to CSV
try:
    df_telangana = pd.DataFrame(data)
    print(f"DataFrame created successfully with {df_telangana.shape[0]} rows.")
    df_telangana.to_csv("telangana_bus_details.csv", index=False)
    print("DataFrame saved to 'telangana_bus_details.csv'")
except ValueError as e:
    print(f"Error creating DataFrame: {e}")

Lengths of all lists:
Bus_names_k: 230
Bus_types_k: 230
Departure_k: 230
Arrival_k: 230
Total_Duration_k: 230
Prices_k: 230
Seats_Available_k: 230
Ratings_k: 33
Route_names: 230
Route_links: 230
Filling missing Ratings data with 'NaN'.
DataFrame created successfully with 230 rows.
DataFrame saved to 'telangana_bus_details.csv'


UTTER PRADESH STATE BUS DETAILS

In [65]:
df8 = pd.read_csv(r"C:\Users\praba\Desktop\red_bus_project\webscrap_busroute_names_links\uttarpradesh.csv")
df8

,Route Name,Route Link
0,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
1,Lucknow to Prayagraj(Uttar Pradesh),https://www.redbus.in/bus-tickets/lucknow-to-a...
2,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
3,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
4,Prayagraj(Uttar Pradesh) to Lucknow,https://www.redbus.in/bus-tickets/allahabad-to...
5,Lucknow to Delhi,https://www.redbus.in/bus-tickets/lucknow-to-d...
6,Lucknow to Agra,https://www.redbus.in/bus-tickets/lucknow-to-agra
7,Delhi to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/delhi-to-ali...
8,Farrukhabad (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/farrukhabad-...
9,Delhi to Farrukhabad (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-far...


In [67]:
# Initialize WebDriver
driver_k = webdriver.Chrome()

# Initialize lists for data storage
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

# Iterate through each row in DataFrame containing route links and names
for i, r in df8.iterrows():
    link = r["Route Link"]
    routes = r["Route Name"]

    # Navigate to the route link
    driver_k.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    try:
        elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)
    except Exception as e:
        print(f"Error clicking elements for {routes}: {e}")
        continue

    # Click "View Buses" button
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except Exception as e:
        print(f"Error clicking 'View Buses' button for {routes}: {e}")
        continue
    time.sleep(2)

    # Scroll to load all buses
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver_k.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except Exception as e:
        print(f"Error extracting ratings for {routes}: {e}")
        rating = []

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

print("Data extraction completed.")
driver_k.quit()

Error clicking 'View Buses' button for Lucknow to Prayagraj(Uttar Pradesh): Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='button']"}
  (Session info: chrome=131.0.6778.206); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF687EF80D5+2992373]
	(No symbol) [0x00007FF687B8BFD0]
	(No symbol) [0x00007FF687A2590A]
	(No symbol) [0x00007FF687A7926E]
	(No symbol) [0x00007FF687A7955C]
	(No symbol) [0x00007FF687AC27D7]
	(No symbol) [0x00007FF687A9F3AF]
	(No symbol) [0x00007FF687ABF584]
	(No symbol) [0x00007FF687A9F113]
	(No symbol) [0x00007FF687A6A918]
	(No symbol) [0x00007FF687A6BA81]
	GetHandleVerifier [0x00007FF687F56A2D+3379789]
	GetHandleVerifier [0x00007FF687F6C32D+3468109]
	GetHandleVerifier [0x00007FF687F60043+3418211]
	GetHandleVerifier [0x00007FF687CEC78B+847787]
	(No symbol) [0x00007FF687B9757F]
	(No s

In [68]:
# Debug: Print the lengths of all lists
print("Lengths of all lists:")
print(f"Bus_names_k: {len(Bus_names_k)}")
print(f"Bus_types_k: {len(Bus_types_k)}")
print(f"Departure_k: {len(Departure_k)}")
print(f"Arrival_k: {len(Arrival_k)}")
print(f"Total_Duration_k: {len(Total_Duration_k)}")
print(f"Prices_k: {len(Prices_k)}")
print(f"Seats_Available_k: {len(Seats_Available_k)}")
print(f"Ratings_k: {len(Ratings_k)}")
print(f"Route_names: {len(Route_names)}")
print(f"Route_links: {len(Route_links)}")

# Ensure all lists have the same length
min_length = min(
    len(Bus_names_k), len(Bus_types_k), len(Departure_k), len(Arrival_k),
    len(Total_Duration_k), len(Prices_k), len(Seats_Available_k),
    len(Route_names), len(Route_links)
)

# Handle missing Ratings data
if len(Ratings_k) < min_length:
    print("Filling missing Ratings data with 'NaN'.")
    Ratings_k.extend([None] * (min_length - len(Ratings_k)))

# Truncate all lists to the same length
Bus_names_k = Bus_names_k[:min_length]
Bus_types_k = Bus_types_k[:min_length]
Departure_k = Departure_k[:min_length]
Arrival_k = Arrival_k[:min_length]
Total_Duration_k = Total_Duration_k[:min_length]
Prices_k = Prices_k[:min_length]
Seats_Available_k = Seats_Available_k[:min_length]
Ratings_k = Ratings_k[:min_length]
Route_names = Route_names[:min_length]
Route_links = Route_links[:min_length]

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    'Seats_Available': Seats_Available_k,
    'Ratings': Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

# Create DataFrame and save to CSV
try:
    df_uttarpradesh = pd.DataFrame(data)
    print(f"DataFrame created successfully with {df_uttarpradesh.shape[0]} rows.")
    df_uttarpradesh.to_csv("uttarpradesh_bus_details.csv", index=False)
    print("DataFrame saved to 'uttarpradesh_bus_details.csv'")
except ValueError as e:
    print(f"Error creating DataFrame: {e}")

Lengths of all lists:
Bus_names_k: 542
Bus_types_k: 542
Departure_k: 542
Arrival_k: 542
Total_Duration_k: 542
Prices_k: 542
Seats_Available_k: 542
Ratings_k: 117
Route_names: 542
Route_links: 542
Filling missing Ratings data with 'NaN'.
DataFrame created successfully with 542 rows.
DataFrame saved to 'uttarpradesh_bus_details.csv'


HARYANA STATE BUS DETAILS

In [69]:
df9 = pd.read_csv(r"C:\Users\praba\Desktop\red_bus_project\webscrap_busroute_names_links\haryana_routes.csv")
df9

,Route Name,Route Link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
2,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
3,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
4,Shimla to Manali,https://www.redbus.in/bus-tickets/shimla-to-ma...
5,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
6,Chandigarh to Manali,https://www.redbus.in/bus-tickets/chandigarh-t...
7,Delhi to Manali,https://www.redbus.in/bus-tickets/delhi-to-manali
8,Manali to Chandigarh,https://www.redbus.in/bus-tickets/manali-to-ch...
9,Dharamshala (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/dharamshala-...


In [71]:
# Initialize WebDriver
driver_k = webdriver.Chrome()

# Initialize lists for data storage
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

# Iterate through each row in DataFrame containing route links and names
for i, r in df9.iterrows():
    link = r["Route Link"]
    routes = r["Route Name"]

    # Navigate to the route link
    driver_k.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    try:
        elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)
    except Exception as e:
        print(f"Error clicking elements for {routes}: {e}")
        continue

    # Click "View Buses" button
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except Exception as e:
        print(f"Error clicking 'View Buses' button for {routes}: {e}")
        continue
    time.sleep(2)

    # Scroll to load all buses
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver_k.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except Exception as e:
        print(f"Error extracting ratings for {routes}: {e}")
        rating = []

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

print("Data extraction completed.")
driver_k.quit()

Error clicking 'View Buses' button for Chamba (Himachal Pradesh) to Chandigarh: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='button']"}
  (Session info: chrome=131.0.6778.206); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF687EF80D5+2992373]
	(No symbol) [0x00007FF687B8BFD0]
	(No symbol) [0x00007FF687A2590A]
	(No symbol) [0x00007FF687A7926E]
	(No symbol) [0x00007FF687A7955C]
	(No symbol) [0x00007FF687AC27D7]
	(No symbol) [0x00007FF687A9F3AF]
	(No symbol) [0x00007FF687ABF584]
	(No symbol) [0x00007FF687A9F113]
	(No symbol) [0x00007FF687A6A918]
	(No symbol) [0x00007FF687A6BA81]
	GetHandleVerifier [0x00007FF687F56A2D+3379789]
	GetHandleVerifier [0x00007FF687F6C32D+3468109]
	GetHandleVerifier [0x00007FF687F60043+3418211]
	GetHandleVerifier [0x00007FF687CEC78B+847787]
	(No symbol) [0x00007FF687B9757F]
	(

In [72]:
# Debug: Print the lengths of all lists
print("Lengths of all lists:")
print(f"Bus_names_k: {len(Bus_names_k)}")
print(f"Bus_types_k: {len(Bus_types_k)}")
print(f"Departure_k: {len(Departure_k)}")
print(f"Arrival_k: {len(Arrival_k)}")
print(f"Total_Duration_k: {len(Total_Duration_k)}")
print(f"Prices_k: {len(Prices_k)}")
print(f"Seats_Available_k: {len(Seats_Available_k)}")
print(f"Ratings_k: {len(Ratings_k)}")
print(f"Route_names: {len(Route_names)}")
print(f"Route_links: {len(Route_links)}")

# Ensure all lists have the same length
min_length = min(
    len(Bus_names_k), len(Bus_types_k), len(Departure_k), len(Arrival_k),
    len(Total_Duration_k), len(Prices_k), len(Seats_Available_k),
    len(Route_names), len(Route_links)
)

# Handle missing Ratings data
if len(Ratings_k) < min_length:
    print("Filling missing Ratings data with 'NaN'.")
    Ratings_k.extend([None] * (min_length - len(Ratings_k)))

# Truncate all lists to the same length
Bus_names_k = Bus_names_k[:min_length]
Bus_types_k = Bus_types_k[:min_length]
Departure_k = Departure_k[:min_length]
Arrival_k = Arrival_k[:min_length]
Total_Duration_k = Total_Duration_k[:min_length]
Prices_k = Prices_k[:min_length]
Seats_Available_k = Seats_Available_k[:min_length]
Ratings_k = Ratings_k[:min_length]
Route_names = Route_names[:min_length]
Route_links = Route_links[:min_length]

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    'Seats_Available': Seats_Available_k,
    'Ratings': Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

# Create DataFrame and save to CSV
try:
    df_haryana = pd.DataFrame(data)
    print(f"DataFrame created successfully with {df_haryana.shape[0]} rows.")
    df_haryana.to_csv("haryana_bus_details.csv", index=False)
    print("DataFrame saved to 'haryana_bus_details.csv'")
except ValueError as e:
    print(f"Error creating DataFrame: {e}")

Lengths of all lists:
Bus_names_k: 437
Bus_types_k: 437
Departure_k: 437
Arrival_k: 437
Total_Duration_k: 437
Prices_k: 437
Seats_Available_k: 437
Ratings_k: 30
Route_names: 437
Route_links: 437
Filling missing Ratings data with 'NaN'.
DataFrame created successfully with 437 rows.
DataFrame saved to 'haryana_bus_details.csv'


SOUTHBENGAL STATE BUS DETAILS

In [76]:
df10 = pd.read_csv(r"C:\Users\praba\Desktop\red_bus_project\webscrap_busroute_names_links\southbengal_routes.csv")
df10

,Route Name,Route Link
0,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...
2,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
3,Haldia to Kolkata,https://www.redbus.in/bus-tickets/haldia-to-ko...
4,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
5,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
6,Kolkata to Arambagh (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
7,Kolkata to Midnapore,https://www.redbus.in/bus-tickets/kolkata-to-m...
8,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
9,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...


In [77]:
# Initialize WebDriver
driver_k = webdriver.Chrome()

# Initialize lists for data storage
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

# Iterate through each row in DataFrame containing route links and names
for i, r in df10.iterrows():
    link = r["Route Link"]
    routes = r["Route Name"]

    # Navigate to the route link
    driver_k.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    try:
        elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)
    except Exception as e:
        print(f"Error clicking elements for {routes}: {e}")
        continue

    # Click "View Buses" button
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except Exception as e:
        print(f"Error clicking 'View Buses' button for {routes}: {e}")
        continue
    time.sleep(2)

    # Scroll to load all buses
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver_k.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except Exception as e:
        print(f"Error extracting ratings for {routes}: {e}")
        rating = []

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

print("Data extraction completed.")
driver_k.quit()

Error clicking 'View Buses' button for Kolkata to Midnapore: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='button']"}
  (Session info: chrome=131.0.6778.206); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF687EF80D5+2992373]
	(No symbol) [0x00007FF687B8BFD0]
	(No symbol) [0x00007FF687A2590A]
	(No symbol) [0x00007FF687A7926E]
	(No symbol) [0x00007FF687A7955C]
	(No symbol) [0x00007FF687AC27D7]
	(No symbol) [0x00007FF687A9F3AF]
	(No symbol) [0x00007FF687ABF584]
	(No symbol) [0x00007FF687A9F113]
	(No symbol) [0x00007FF687A6A918]
	(No symbol) [0x00007FF687A6BA81]
	GetHandleVerifier [0x00007FF687F56A2D+3379789]
	GetHandleVerifier [0x00007FF687F6C32D+3468109]
	GetHandleVerifier [0x00007FF687F60043+3418211]
	GetHandleVerifier [0x00007FF687CEC78B+847787]
	(No symbol) [0x00007FF687B9757F]
	(No symbol) [0x00007

In [78]:
# Debug: Print the lengths of all lists
print("Lengths of all lists:")
print(f"Bus_names_k: {len(Bus_names_k)}")
print(f"Bus_types_k: {len(Bus_types_k)}")
print(f"Departure_k: {len(Departure_k)}")
print(f"Arrival_k: {len(Arrival_k)}")
print(f"Total_Duration_k: {len(Total_Duration_k)}")
print(f"Prices_k: {len(Prices_k)}")
print(f"Seats_Available_k: {len(Seats_Available_k)}")
print(f"Ratings_k: {len(Ratings_k)}")
print(f"Route_names: {len(Route_names)}")
print(f"Route_links: {len(Route_links)}")

# Ensure all lists have the same length
min_length = min(
    len(Bus_names_k), len(Bus_types_k), len(Departure_k), len(Arrival_k),
    len(Total_Duration_k), len(Prices_k), len(Seats_Available_k),
    len(Route_names), len(Route_links)
)

# Handle missing Ratings data
if len(Ratings_k) < min_length:
    print("Filling missing Ratings data with 'NaN'.")
    Ratings_k.extend([None] * (min_length - len(Ratings_k)))

# Truncate all lists to the same length
Bus_names_k = Bus_names_k[:min_length]
Bus_types_k = Bus_types_k[:min_length]
Departure_k = Departure_k[:min_length]
Arrival_k = Arrival_k[:min_length]
Total_Duration_k = Total_Duration_k[:min_length]
Prices_k = Prices_k[:min_length]
Seats_Available_k = Seats_Available_k[:min_length]
Ratings_k = Ratings_k[:min_length]
Route_names = Route_names[:min_length]
Route_links = Route_links[:min_length]

# Create the DataFrame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    'Seats_Available': Seats_Available_k,
    'Ratings': Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

# Create DataFrame and save to CSV
try:
    df_southbengal = pd.DataFrame(data)
    print(f"DataFrame created successfully with {df_southbengal.shape[0]} rows.")
    df_southbengal.to_csv("southbengal_bus_details.csv", index=False)
    print("DataFrame saved to 'southbengal_bus_details.csv'")
except ValueError as e:
    print(f"Error creating DataFrame: {e}")

Lengths of all lists:
Bus_names_k: 371
Bus_types_k: 371
Departure_k: 371
Arrival_k: 371
Total_Duration_k: 371
Prices_k: 371
Seats_Available_k: 371
Ratings_k: 66
Route_names: 371
Route_links: 371
Filling missing Ratings data with 'NaN'.
DataFrame created successfully with 371 rows.
DataFrame saved to 'southbengal_bus_details.csv'
